In [1]:
from pathlib import Path
import pandas as pd
from bitstruct import unpack_from as upf

In [25]:
DIR = Path(r"C:\Users\ucasbwh\Desktop\Proc Dev")

HKfiles = sorted(DIR.rglob("*HK.txt"))
print(HKfiles)

[]


In [3]:
reading.stem

NameError: name 'reading' is not defined

In [0]:
reading = HKfiles[-1]
log_data = open(reading, 'r')
#columns = ['Text', 'TS_Byte', 'RAW']
DT = pd.read_table(reading, sep=']', header=None)
DT.head()

In [0]:
DL = pd.DataFrame()
DL['SPW_RAW'] = DT[1].apply(lambda x: x.replace('0x','').replace(' ',''))
DL.head()

In [0]:
DL['HK_RAW'] = DL.SPW_RAW.apply(lambda x: x[108-84:-2])

In [0]:
DT['Unix'] = DT[0].apply(lambda x: x[11:-12])

In [0]:
DL['DT'] = pd.to_datetime(DT['Unix'], unit='ms')
DL.head()

In [0]:
Bin = DL['HK_RAW'].apply(lambda x: bytearray.fromhex(x))
Bin.head()

In [0]:
## Test 1 Block Type is always 0
Extract = Bin.apply(lambda x: upf('u1', x, offset=0)[0]).astype('int64')
if True in (Extract !=0).unique():
    print("Fail")

In [0]:
## Test 2 PanCam ID is always 5
Extract = Bin.apply(lambda x: upf('u4', x, offset=4)[0]).astype('int64')
if True in (Extract !=5).unique():
    print("Fail")

In [0]:
## Test 3 Reserved Byte is always 0
Extract = Bin.apply(lambda x: upf('u8', x, offset=11*8)[0]).astype('int64')
if True in (Extract !=0).unique():
    print("Fail")

In [27]:
DIR = Path(r"C:\Users\ucasbwh\Desktop\Proc Dev")

HKfiles = sorted(DIR.rglob("*Unproc_HKTM.pickle"))
print(HKfiles)

RTM = pd.read_pickle(HKfiles[0])

[WindowsPath('C:/Users/ucasbwh/Desktop/Proc Dev/191106_212023_Unproc_HKTM.pickle')]


In [0]:
RTM

In [0]:
Bin = RTM['RAW'].apply(lambda x: bytearray.fromhex(x))

In [0]:
def PandUPF(Column, Len, OffBy, OffBi):
    """Extracts a single RAW value from a binary pandas data column"""
    if int(Len[1:]) > 63:
        raise ValueError("PandUPF used for variable larger than 63 bits. Returned value is cast to an Int64")
    Extract = Column.apply(lambda x: upf(Len, x, offset=8*OffBy+OffBi)[0]).astype('Int64')
    return Extract

In [0]:
TM = pd.DataFrame()
TM['TM_Type_ID']     = PandUPF(Bin, 'u6', 0, 8)
TM

In [0]:
NEBin = Bin[TM['TM_Type_ID']==1]
display(NEBin)

In [0]:
TM['TM_Type_ID'].head()

In [0]:
Bin.head()

In [0]:
NEBin.head()

In [0]:
NEBin[82:83]

In [0]:
## Test 4 Reserved Byte is always 0
Extract = Bin.apply(lambda x: upf('u16', x, offset=12*8)[0]).astype('int64')
if True in (Extract !=1801).unique():
    print("Fail")